In this notebook we compute a series of metrics from the scaling and root scaling models

In [1]:
import logging

import pandas as pd
import numpy as np
import math
from typing import Optional, List, Tuple
import torch

from sdo.viz.compare_models import load_pred_and_gt
from sdo.metrics.ssim_metric import ssim

In [2]:
def load_pred_and_gt_no_mask(results_path: str, revert_root: bool = False, 
                     frac: Optional[float]=1.0) -> Tuple[np.array, np.array]:
    """
    Load predictions and ground truths from file, optionally remove
    root scaling,
    Args:
        results_path: path to file containing gt and predictions in npz format
        revert_root: if True both predictions and ground truth are **2
        frac: percentage of samples to be loaded, selected sequentially

    Returns:
        Y_test, Y_pred
    """
    Y = np.load(results_path)
    shape = Y.shape
    Y_test = Y[:, :, 0:int(shape[2] / 2), :]
    Y_pred = Y[:, :, int(shape[2] / 2):, :]
    #print(f"Shape Y {shape}")

    if revert_root:
        logging.info('Reverting root scaling')
        Y_test = np.power(Y_test, 2)
        Y_pred = np.power(Y_pred, 2)

    if frac< 1.0:
        Y_size = shape[0]
        sample_size = int(Y_size * frac)
        Y_test = Y_test[0:sample_size, :, :, :]
        Y_pred = Y_pred[0:sample_size, :, :, :]
        
    nsamples = Y_test.shape[0]
    
    Y_test = Y_test.reshape(nsamples, 1, 512, 512)
    Y_pred = Y_pred.reshape(nsamples, 1, 512, 512)
    return Y_test, Y_pred

In [3]:
results_path = '/fdl_sdo_data/bucket2/EXPERIMENT_RESULTS/VIRTUAL_TELESCOPE'
dict_exp = {
    '211': [
    [ '/vale_exp_23/0600_vale_exp_23_test_predictions.npy', True, '211_root'],
    ['/vale_exp_20/0600_vale_exp_20_test_predictions.npy', False, '211'],
    ],
    '193': [
    ['/vale_exp_25/0600_vale_exp_25_test_predictions.npy', True, '193_root'],
    ['/vale_exp_13bis/0600_vale_exp_13bis_test_predictions.npy', False, '193'],
    ],
    '171': [
    [ '/vale_exp_26/0600_vale_exp_26_test_predictions.npy', True, '171_root'],
    ['/vale_exp_14bis/0600_vale_exp_14bis_test_predictions.npy', False, '171']
    ],
    '094': [
        [ '/vale_exp_27/0400_vale_exp_27_test_predictions.npy', True, '094_root'],
        ['/vale_exp_18/0600_vale_exp_18_test_predictions.npy', False, '094'],
    ]
}
#use 1.0 for final results, smaller value for testing
frac = 1.0

In [4]:
df_results = pd.DataFrame()
for key in {'211'}:
    print(f'Channel {key}')
    for exp in dict_exp[key]:
        print(f'Experiment {exp[0]}')
        Y_test, Y_pred = load_pred_and_gt_no_mask(results_path + exp[0], revert_root=exp[1], frac=frac)
        mse = np.square(np.subtract(Y_test, Y_pred)).mean() 

Channel 211
Experiment /vale_exp_23/0600_vale_exp_23_test_predictions.npy
Experiment /vale_exp_20/0600_vale_exp_20_test_predictions.npy


In [8]:
np.square(np.subtract(Y_test, Y_pred)).mean()

0.0023111086

In [9]:
math.sqrt(np.square(np.subtract(Y_test, Y_pred)).mean())

0.048073991241375486

In [10]:
math.sqrt(np.square(np.subtract(Y_test, Y_pred)).mean())/Y_test.mean()

0.18228568998252279

In [12]:
mse = np.square(np.subtract(Y_test, Y_pred)).mean()
nmse = mse / np.square(Y_test).mean()
nmse2 = np.square(np.subtract(Y_test, Y_pred)).sum()/np.square(Y_test).sum()
mse, nmse, nmse2

(0.0023111086, 0.008748424, 0.008748424)

In [16]:
df_results = pd.DataFrame()
for key in dict_exp:
    print(f'Channel {key}')
    for exp in dict_exp[key]:
        print(f'Experiment {exp[0]}')
        Y_test, Y_pred = load_pred_and_gt_no_mask(results_path + exp[0], revert_root=exp[1], frac=frac)
        mse = np.square(np.subtract(Y_test, Y_pred)).mean()
        rmse = math.sqrt(mse)
        nmse = mse / np.square(Y_test).mean()
        nrmse = rmse / Y_test.mean()
        # ssim is expecting images not arrays
        # array of size 1748467 -> the function load_pred_and_gt apply mask on the array, 
        # we need instead to reshape first and, if we want, appy the mask before the computation
        # modify the function. Better to compute metrics with and without mask
        #ssim = np.abs(1-ssim(Y_pred.reshape((667, 512, 512)), Y_test.reshape((16, 512, 512))))
        t_Y_pred = torch.from_numpy(Y_pred)
        t_Y_test = torch.from_numpy(Y_test)
        val_ssim = torch.abs(1-ssim(t_Y_pred, t_Y_test))
        pm = (nrmse + val_ssim.item())/2
        mean_test = Y_test.mean()
        df_results[exp[2]] = [nmse, nrmse, val_ssim.item(), pm, mean_test]

Channel 211
Experiment /vale_exp_23/0600_vale_exp_23_test_predictions.npy
Experiment /vale_exp_20/0600_vale_exp_20_test_predictions.npy
Channel 193
Experiment /vale_exp_25/0600_vale_exp_25_test_predictions.npy
Experiment /vale_exp_13bis/0600_vale_exp_13bis_test_predictions.npy
Channel 171
Experiment /vale_exp_26/0600_vale_exp_26_test_predictions.npy
Experiment /vale_exp_14bis/0600_vale_exp_14bis_test_predictions.npy
Channel 094
Experiment /vale_exp_27/0400_vale_exp_27_test_predictions.npy
Experiment /vale_exp_18/0600_vale_exp_18_test_predictions.npy


In [17]:
df_results.index = ['NMSE', 'NRMSE', 'SSIM', '(NRMSE + SSIM)/2', 'Mean']
df_results

,211_root,211,193_root,193,171_root,171,094_root,094
NMSE,0.010024,0.008748,0.013414,0.013015,0.015270,0.010151,0.009482,0.013643
NRMSE,0.195127,0.182286,0.225717,0.222332,0.240829,0.196360,0.189773,0.227641
SSIM,0.040844,0.046189,0.022866,0.024522,0.030636,0.034892,0.114447,0.138455
(NRMSE + SSIM)/2,0.117985,0.114237,0.124292,0.123427,0.135732,0.115626,0.152110,0.183048
Mean,0.263729,0.263729,0.087910,0.087910,0.131864,0.131864,26.372948,26.372839


In [15]:
print(df_results.to_latex(index=True)) 

\begin{tabular}{lrrrrrrrr}
\toprule
{} &  211\_root &       211 &  193\_root &       193 &  171\_root &       171 &  094\_root &       094 \\
\midrule
NMSE             &  0.010024 &  0.008748 &  0.013414 &  0.013015 &  0.015270 &  0.010151 &  0.009482 &  0.013643 \\
NRMSE            &  0.195127 &  0.182286 &  0.225717 &  0.222332 &  0.240829 &  0.196360 &  0.189773 &  0.227641 \\
SSIM             &  0.040844 &  0.046189 &  0.022866 &  0.024522 &  0.030636 &  0.034892 &  0.114447 &  0.138455 \\
(NRMSE + SSIM)/2 &  0.117985 &  0.114237 &  0.124292 &  0.123427 &  0.135732 &  0.115626 &  0.152110 &  0.183048 \\
\bottomrule
\end{tabular}



In [ ]:
df_results = pd.DataFrame()
for key in dict_exp:
    print(f'Channel {key}')
    for exp in dict_exp[key]:
        print(f'Experiment {exp[0]}')
        Y_test, Y_pred = load_pred_and_gt_no_mask(results_path + exp[0], revert_root=exp[1], frac=frac)
        mse = np.square(np.subtract(Y_test, Y_pred)).mean() 
        rmse = math.sqrt(mse)
        # ssim is expecting images not arrays
        # array of size 1748467 -> the function load_pred_and_gt apply mask on the array, 
        # we need instead to reshape first and, if we want, appy the mask before the computation
        # modify the function. Better to compute metrics with and without mask
        #ssim = np.abs(1-ssim(Y_pred.reshape((667, 512, 512)), Y_test.reshape((16, 512, 512))))
        t_Y_pred = torch.from_numpy(Y_pred)
        t_Y_test = torch.from_numpy(Y_test)
        val_ssim = torch.abs(1-ssim(t_Y_pred, t_Y_test))
        pm = (rmse + val_ssim.item())/2
        df_results[exp[2]] = [mse, rmse, val_ssim.item(), pm]

In [7]:
df_results.index = ['MSE', 'RMSE', 'SSIM', '(RMSE + SSIM)/2']
df_results

,211_root,211,193_root,193,171_root,171,094_root,094
MSE,0.002648,0.002311,0.000394,0.000382,0.001008,0.000670,25.048731,36.042522
RMSE,0.051461,0.048074,0.019843,0.019545,0.031757,0.025893,5.004871,6.003542
SSIM,0.040844,0.046189,0.022866,0.024522,0.030636,0.034892,0.114447,0.138455
(RMSE + SSIM)/2,0.046152,0.047131,0.021355,0.022033,0.031196,0.030392,2.559659,3.070999


In [9]:
print(df_results.to_latex(index=True)) 

\begin{tabular}{lrrrrrrrr}
\toprule
{} &  211\_root &       211 &  193\_root &       193 &  171\_root &       171 &   094\_root &        094 \\
\midrule
MSE             &  0.002648 &  0.002311 &  0.000394 &  0.000382 &  0.001008 &  0.000670 &  25.048731 &  36.042522 \\
RMSE            &  0.051461 &  0.048074 &  0.019843 &  0.019545 &  0.031757 &  0.025893 &   5.004871 &   6.003542 \\
SSIM            &  0.040844 &  0.046189 &  0.022866 &  0.024522 &  0.030636 &  0.034892 &   0.114447 &   0.138455 \\
(RMSE + SSIM)/2 &  0.046152 &  0.047131 &  0.021355 &  0.022033 &  0.031196 &  0.030392 &   2.559659 &   3.070999 \\
\bottomrule
\end{tabular}



# Same as above but for linear model

In [3]:
results_path = '/fdl_sdo_data/bucket/EXPERIMENT_RESULTS/VIRTUAL_TELESCOPE'
dict_exp = {
    '211': [
    ['/vale_exp_linear_20/0600_vale_exp_linear_20_test_predictions.npy', False, '211'],
    ],
    '193': [
    ['/vale_exp_linear_13/0600_vale_exp_linear_13_test_predictions.npy', False, '193'],
    ],
    '171': [
    ['/vale_exp_linear_14/0600_vale_exp_linear_14_test_predictions.npy', False, '171']
    ],
    '094': [
        ['/vale_exp_linear_18/0600_vale_exp_linear_18_test_predictions.npy', False, '094'],
    ]
}
#use 1.0 for final results, smaller value for testing
frac = 1.0

In [4]:
df_results = pd.DataFrame()
for key in dict_exp:
    print(f'Channel {key}')
    for exp in dict_exp[key]:
        print(f'Experiment {exp[0]}')
        Y_test, Y_pred = load_pred_and_gt_no_mask(results_path + exp[0], revert_root=exp[1], frac=frac)
        mse = np.square(np.subtract(Y_test, Y_pred)).mean() 
        rmse = math.sqrt(mse)
        # ssim is expecting images not arrays
        # array of size 1081344 -> the function load_pred_and_gt apply mask on the array, 
        # we need instead to reshape first and, if we want, appy the mask before the computation
        # modify the function. Better to compute metrics with and without mask
        #ssim = np.abs(1-ssim(Y_pred.reshape((667, 512, 512)), Y_test.reshape((16, 512, 512))))
        t_Y_pred = torch.from_numpy(Y_pred)
        t_Y_test = torch.from_numpy(Y_test)
        val_ssim = torch.abs(1-ssim(t_Y_pred, t_Y_test))
        pm = (rmse + val_ssim.item())/2
        df_results[exp[2]] = [mse, rmse, val_ssim.item(), pm]

Channel 211
Experiment /vale_exp_linear_20/0600_vale_exp_linear_20_test_predictions.npy
Channel 193
Experiment /vale_exp_linear_13/0600_vale_exp_linear_13_test_predictions.npy
Channel 171
Experiment /vale_exp_linear_14/0600_vale_exp_linear_14_test_predictions.npy
Channel 094
Experiment /vale_exp_linear_18/0600_vale_exp_linear_18_test_predictions.npy


In [20]:
Y = np.load('/fdl_sdo_data/bucket/EXPERIMENT_RESULTS/VIRTUAL_TELESCOPE/vale_exp_linear_20/0600_vale_exp_linear_20_test_predictions.npy')
shape = Y.shape
Y_test = Y[:, :, 0:int(shape[2] / 2), :]
Y_pred = Y[:, :, int(shape[2] / 2):, :]

In [22]:
print(f"Shape Y {shape}")

Shape Y (667, 1, 256, 128)


In [5]:
df_results.index = ['MSE', 'RMSE', 'SSIM', '(RMSE + SSIM)/2']
df_results

,211,193,171,094
MSE,0.198024,0.021804,0.048970,2312.233887
RMSE,0.444998,0.147662,0.221291,48.085693
SSIM,0.588910,0.441623,0.490644,0.976495
(RMSE + SSIM)/2,0.516954,0.294643,0.355968,24.531094


In [6]:
df_results = pd.DataFrame()
for key in dict_exp:
    print(f'Channel {key}')
    for exp in dict_exp[key]:
        print(f'Experiment {exp[0]}')
        Y_test, Y_pred = load_pred_and_gt_no_mask(results_path + exp[0], revert_root=exp[1], frac=frac)
        mse = np.square(np.subtract(Y_test, Y_pred)).mean()
        rmse = math.sqrt(mse)
        nmse = mse / np.square(Y_test).mean()
        nrmse = rmse / Y_test.mean()
        # ssim is expecting images not arrays
        # array of size 1748467 -> the function load_pred_and_gt apply mask on the array, 
        # we need instead to reshape first and, if we want, appy the mask before the computation
        # modify the function. Better to compute metrics with and without mask
        #ssim = np.abs(1-ssim(Y_pred.reshape((667, 512, 512)), Y_test.reshape((16, 512, 512))))
        t_Y_pred = torch.from_numpy(Y_pred)
        t_Y_test = torch.from_numpy(Y_test)
        val_ssim = torch.abs(1-ssim(t_Y_pred, t_Y_test))
        pm = (nrmse + val_ssim.item())/2
        mean_test = Y_test.mean()
        df_results[exp[2]] = [nmse, nrmse, val_ssim.item(), pm, mean_test]

Channel 211
Experiment /vale_exp_linear_20/0600_vale_exp_linear_20_test_predictions.npy
Channel 193
Experiment /vale_exp_linear_13/0600_vale_exp_linear_13_test_predictions.npy
Channel 171
Experiment /vale_exp_linear_14/0600_vale_exp_linear_14_test_predictions.npy
Channel 094
Experiment /vale_exp_linear_18/0600_vale_exp_linear_18_test_predictions.npy


In [11]:
df_results.index = ['NMSE', 'NRMSE', '1 - SSIM', '(NRMSE + |1 - SSIM|)/2', 'Mean']
df_results

,211,193,171,094
NMSE,0.749594,0.742833,0.741476,0.875264
NRMSE,1.687336,1.679708,1.678174,1.823300
1 - SSIM,0.588910,0.441623,0.490644,0.976495
(NRMSE + |1 - SSIM|)/2,1.138123,1.060665,1.084409,1.399897
Mean,0.263728,0.087910,0.131864,26.372892
